## Notebook Content

This notebook for autorun the built webpage __(Calory Predictor)__ to show the predicted calory against the true calory.

## Import Libraries

In [1]:
from bs4 import BeautifulSoup
import sqlite3 as sql
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
from flask import Response
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from sklearn.ensemble import GradientBoostingRegressor

## Read the Used Data

In [2]:
Xs_test_connection= sql.connect('/Users/shahadsulaiman/Desktop/Linear-Regression-detect-calories/Xs_test_FE.db')
Xs_test = pd.read_sql('SELECT * FROM Xs_test_FE', Xs_test_connection)

In [3]:
Xs_val_train_connection = sql.connect('/Users/shahadsulaiman/Desktop/Linear-Regression-detect-calories/Xs_train_val_FE.db')
Xs_val_train = pd.read_sql('SELECT * FROM Xs_train_val_FE', Xs_val_train_connection)

In [4]:
y_val_train_connection= sql.connect('/Users/shahadsulaiman/Desktop/Linear-Regression-detect-calories/y_train_val_FE.db')
y_val_train = pd.read_sql('SELECT * FROM y_train_val_FE', y_val_train_connection)

In [5]:
y_test_connection= sql.connect('/Users/shahadsulaiman/Desktop/Linear-Regression-detect-calories/y_test.db')
y_test = pd.read_sql('SELECT * FROM y_test', y_test_connection)

In [6]:
Xs_test.drop('index', axis='columns', inplace = True)
y_test.drop('index', axis='columns', inplace = True)
Xs_val_train.drop('index', axis='columns', inplace = True)
y_val_train.drop('index', axis='columns', inplace = True)

## Model Initialization 

In [7]:
model = GradientBoostingRegressor(random_state = 100)
fitting = model.fit(Xs_val_train, y_val_train.values.ravel())

this for take 10 sample randomly from  testsets and use it for webpage auto run

In [8]:
y_smaple = y_test.sample(5, random_state = 4 )
Xs_sample = Xs_test.sample(5, random_state = 4 )

show the samples

In [9]:
y_smaple

,CALORY
63,140.0
34,100.0
184,25.0
146,200.0
130,110.0


In [10]:
Xs_sample

,TOTAL_FAT,SATURATED_FAT,TOTAL_CARBOHYDRATE,PROTEIN,CALCIUM,IRON,POTASSIUM,SUGARS,FIBER,SATURATED_FAT+TOTAL_FAT,TOTAL_CARBOHYDRATE_SQ
63,0.265267,-0.154489,0.255549,-0.185370,-1.002445,-0.922115,-1.158363,-0.508651,-0.187875,0.110778,0.065305
34,0.437010,1.069709,-1.859569,1.141709,-1.002445,-0.922115,-1.158363,-1.211867,-1.325534,1.506719,3.457996
184,-1.559334,-1.255148,-1.102616,-0.708747,-1.002445,-0.922115,-1.158363,-1.211867,-1.325534,-2.814482,1.215761
146,0.437010,0.621680,0.654732,0.335351,0.737543,1.476726,0.962948,-0.508651,0.319771,1.058690,0.428674
130,0.437010,1.461936,-0.850283,-0.185370,0.818139,-0.922115,0.191915,0.867817,0.460151,1.898946,0.722982


## Web Driver Initialization

In [11]:
#Amjad Path
path = "/Users/amjad/Downloads/chromedriver"

In [12]:
#Shahad Path
path = "/Users/shahadsulaiman/Downloads/chromedriver"

In [13]:
chromedriver = path # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [14]:
driver = webdriver.Chrome(chromedriver) ## note that this webpage file is present localy
driver.get('file:////Users/shahadsulaiman/Desktop/Linear-Regression-detect-calories/Webpage_for_auto_test/CaloryPredictor.html')

## Start Auto Run

In [15]:
## Initialize each text input to write the smaples features inside 
Total_Fat_in = driver.find_element_by_id("Total_Fat")
Saturated_Fat_in = driver.find_element_by_id("Saturated_Fat")
Total_Carbohydrate_in = driver.find_element_by_id("Total_Carbohydrate")
Protein_in = driver.find_element_by_id("Protein")
Calcium_in = driver.find_element_by_id("Calcium")
Iron_in = driver.find_element_by_id("Iron")
Sugars_in = driver.find_element_by_id("Sugars")
Fiber_in = driver.find_element_by_id("Fiber")
inputs_list = [Total_Fat_in, Saturated_Fat_in, Total_Carbohydrate_in,  ## store them into a list
               Protein_in, Calcium_in, Iron_in, Sugars_in, Fiber_in]

In [16]:
y_preds_in = [] ## list to store text inputs that will hold the results of predected y
y_true_in = [] ## list to store text inputs that will hold the true y

for i in range(5): ## this loop will start append each text input in it's right list
    d = driver.find_element_by_id(f"DOutput-{str(i+1)}")
    t = driver.find_element_by_id(f"TOutput-{str(i+1)}")
    y_preds_in.append(d)
    y_true_in.append(t)

In [17]:
def fill_inputs(row):
    """
    This method take only one sample and start filling the text inputs with them
    """
    for i, input_ in enumerate(inputs_list):
        input_.send_keys(str(row[0][i]))

In [18]:
def clear_inputs():
    """
    This method will clear all text inputs
    """
    for input_ in inputs_list:
        input_.clear()

This below for loop code will perform like the following: loop over samples and call method __fill_inputs__ to fill them into text inputs then pause for 1 second then predict the y and display it with the true y then pause for 1 second, then call __clear_inputs__ method to clear all inputs and be ready for the next iteration then pause 2 seconds before the next iteration start.

In [19]:
for i in range(5):
    X = Xs_sample.iloc[i,:].values.reshape(1, -1)
    y = y_smaple.iloc[i].values[0]
    fill_inputs(X)
    time.sleep(1)
    y_pred = fitting.predict(X)
    y_preds_in[i].send_keys(str('{:.2f}'.format(y_pred[0])))
    y_true_in[i].send_keys(str(y))
    time.sleep(1)
    clear_inputs()
    time.sleep(2)